In [1]:
import torch
import argparse
import torch.nn as nn
import torch.optim as optim
import time

from tqdm.auto import tqdm

from model import build_model
from datasets import get_datasets, get_data_loaders
from utils import save_model, save_plots
from torch.utils.mobile_optimizer import optimize_for_mobile

/home/edabk/anaconda3/envs/yolov7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:





device = 'cpu'
print(f"Computation device: {device}")
dataset_train, dataset_valid, dataset_classes = get_datasets(False)
model = build_model(
    pretrained=False, 
    fine_tune=True, 
    num_classes=len(dataset_classes)
)
model = model.to(device)
    

Computation device: cpu
[INFO]: Not loading pre-trained weights
[INFO]: Fine-tuning all layers...


/home/edabk/anaconda3/envs/yolov7/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/edabk/anaconda3/envs/yolov7/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
checkpoint = torch.load("/home/edabk/quangnghiem/samsung_antifake/last_effb0.pt")

In [12]:
checkpoint

OrderedDict([('features.0.0.weight',
              tensor([[[[ 1.2156e-01,  6.5634e-01,  4.5671e-01],
                        [-1.1092e-01, -6.1004e-01, -3.3345e-01],
                        [ 2.7964e-02, -1.0312e-01, -1.0324e-01]],
              
                       [[ 6.3553e-02,  1.6552e+00,  1.7436e+00],
                        [-1.3646e-01, -1.5367e+00, -1.5937e+00],
                        [ 5.0196e-02, -1.1360e-01, -1.2600e-01]],
              
                       [[ 8.7276e-02,  3.6126e-01,  2.6946e-01],
                        [-1.1966e-01, -2.8122e-01, -2.1883e-01],
                        [ 3.6658e-02, -7.0751e-02, -8.1917e-02]]],
              
              
                      [[[ 1.6449e-01, -2.0041e-01,  8.3092e-02],
                        [ 8.9009e-01, -1.2110e+00,  2.7610e-01],
                        [ 1.0740e+00, -1.2603e+00,  2.0645e-01]],
              
                       [[ 3.2816e-01, -4.3449e-01,  1.8769e-01],
                        [ 1.6213e+00, 

In [5]:
if "model_state_dict" in checkpoint:
    temp = checkpoint["model_state_dict"].copy()
    for key in temp:
        new_key = key.replace("module.", "")
        checkpoint["model_state_dict"][new_key] = checkpoint["model_state_dict"].pop(key)
    model.load_state_dict(checkpoint["model_state_dict"])
else:
    model.load_state_dict(checkpoint)

In [6]:

model = model.to(device)

In [7]:
for params in model.parameters():
    if 'cuda' in  str(params.device):
        print(params)

In [8]:
model.eval()

scripted_module = torch.jit.script(model)

scripted_module.save("./export_models/effb0.pt")

scripted_module._save_for_lite_interpreter("./export_models/effb0.ptl")

In [9]:
torch.save(model.state_dict(), "./export_models/last_effb0.pt")

In [10]:
scripted_module.to(torch.device("cpu"))
optimized_torchscript = optimize_for_mobile(scripted_module)
optimized_torchscript._save_for_lite_interpreter("./export_models/effb0_script.ptl")

In [11]:
from torch.utils.mobile_optimizer import optimize_for_mobile

model = model.to(torch.device("cpu"))
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
optimized_traced_model = optimize_for_mobile(traced_script_module)
optimized_traced_model._save_for_lite_interpreter("./export_models/effb0_optim.ptl")